In [ ]:
# asked for a breakdown into smaller chunks

# !pip install langgraph langchain-openai tavily-python pydantic
# all installed already

In [2]:
# imports, modules
import os
from typing import Literal, Optional, TypedDict

from IPython.display import Image, display
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph
from langgraph.graph.state import CompiledStateGraph
from tavily import TavilyClient


In [ ]:
# load environment variables - api keys, and set other variables
from dotenv import load_dotenv

load_dotenv()

REPUTABLE = ["nhs.uk","who.int","cdc.gov","medlineplus.gov","nih.gov","mayoclinic.org"]


True

In [4]:
# State
class HealthState(TypedDict, total=False):
    topic: Optional[str]
    human_input: Optional[str]
    next_action: Optional[Literal["ask_topic","echo"]]

In [ ]:
# functions
def ask_topic(state: HealthState) -> HealthState:
    # In a notebook, you'll set state["human_input"] manually, then invoke.
    topic = (state.get("human_input") or "").strip()
    return {"topic": topic, "human_input": None, "next_action": "echo"}

def echo(state: HealthState) -> HealthState:
    print(f"Topic captured: {state.get('topic')!r}")
    return {}  # finished

def search(state: HealthState) -> HealthState:
    topic = state["topic"]
    query = f"{topic} site:({' OR '.join(REPUTABLE)})"
    res = tavily.search(query=query, include_raw_content=False, max_results=5)
    hits = [r for r in res.get("results", []) if any(d in r.get("url","") for d in REPUTABLE)]
    print(f"Found {len(hits)} reputable results")
    return {"hits": hits, "next_action": "done"}

In [ ]:
# workflow
workflow = StateGraph(HealthState)

workflow.add_node("ask_topic", ask_topic)
workflow.add_node("search", search)

workflow.add_edge(START, "ask_topic")
workflow.add_edge("ask_topic", "search")
workflow.add_edge("search", END)
workflow.set_entry_point("ask_topic")

app = workflow.compile(checkpointer=MemorySaver())

MemoryConfig = {"configurable": {"thread_id": "notebook-1"}}

In [11]:
# --- Run step-by-step in the notebook ---
state: HealthState = {"next_action": "ask_topic"}   # initial state


In [12]:
state["human_input"] = "hypertension"     # simulate user input


In [13]:
state = app.invoke(state, config=MemoryConfig)                 # runs ask_topic -> echo

Topic captured: 'hypertension'
